# A/B Testing Project

## Introduction

This analysis analyzes an A/B test launched by analysts for an international online store. The purpose and aim of the analysis is to examine the test data and see whether it was carried out correctly. Then, to analyze the results.

The technical description of the test they launched is as follows: 
- The test's name: "recommender_system_test";
- Experiment groups: A (control), B (new payment funnel);
- The test's launch date: 2020-12-07;
- Last date of taking up new users: 2020-12-21;
- Test end date: 2021-01-01;
- The test's audience: 15% of the new users from the EU region;
- The purpose of the test: testing changes related to the introduction of an improved recommendation system;
- The test's expected result: within 14 days of signing up, users will show better conversion into product page views (the 'product_page' event), product card views ('product_card') and purchases ('purchase'). At each of the stage of the funnel 'product_page → product_card → purchase', there will be at least a 10% increase;
- The expected number of test participants: 6,000.

The aim of this analysis: **examine the test data and see whether it was carried out correctly. Then, analyze the results.**

## Description of the Data

In this analysis, we have four datasets: 
- marketing_events — the calendar of marketing events for 2020;
- users — all users who signed up in the online store from December 7 to 21, 2020;
- user_event — all events of the new users within the period from December 7, 2020 to January 1, 2021;
- participants — table containing test participants.

Hereinafter, I will detail the data within each of the datasets note above:
1. marketing_events:
    - 'name' - the name of the marketing event;
    - 'regions' - regions where the ad campaign will be held;
    - 'start_dt' - campaign start date;
    - 'finish_dt' - campaign end date.


2. users:
    - user_id - each user's unique user id;
    - first_date - sign-up date;
    - region - the user's region;
    - device - device used to sign up.


3. user_events:
    - user_id - each user's unique user id;
    - event_dt - event date and time;
    - event_name - event type name;
    - details - additional data on the event (for instance, the order total in USD for 'purchase' events).


4. participants:
    - user_id - each user's unique user id;
    - ab_test - test name;
    - group - the test group the user belonged to.

## Loading and Initial Exploration of the Data

In [1]:
# importing library to ignore jupyter warnings 

import warnings
warnings.filterwarnings('ignore')

In [2]:
# importing used libaraies 

import matplotlib
import pandas as pd
import math as mth
import numpy as np
import datetime as dt
import seaborn as sns
import plotly.express as px
from scipy import stats as st
from scipy.stats import levene
import matplotlib.pyplot as plt
from plotly import graph_objects as go
from IPython.display import display_html
from plotly.subplots import make_subplots

#unused libraries: 

#import random
#import sidetable
#from random import sample
#from functools import reduce
#from itertools import chain,cycle
#from sklearn.cluster import KMeans
#from sklearn.metrics import roc_auc_score
#from sklearn.tree import DecisionTreeClassifier
#from sklearn.preprocessing import StandardScaler
#from sklearn.model_selection import train_test_split
#from scipy.cluster.hierarchy import dendrogram, linkage
#from sklearn.linear_model import Lasso, Ridge, LogisticRegression
#from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
#from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, 
#accuracy_score, precision_score, recall_score, f1_score


In [3]:
#loading of the data frames and saving them in variables

try: 
    marketing_events = pd.read_csv(
        '/Users/doronerlich/Desktop/Code ME/Final Project/ab_project_marketing_events_us.csv')
    users = pd.read_csv('/Users/doronerlich/Desktop/Code ME/Final Project/final_ab_new_users_upd_us.csv', 
                        dtype={'device': 'category', 'region': 'category'})
    user_events = pd.read_csv('/Users/doronerlich/Desktop/Code ME/Final Project/final_ab_events_upd_us.csv', 
                              parse_dates=['event_dt'])
    participants = pd.read_csv('/Users/doronerlich/Desktop/Code ME/Final Project/final_ab_participants_upd_us.csv')
except:
    marketing_events = pd.read_csv('/datasets/ab_project_marketing_events_us.csv')
    users = pd.read_csv('/datasets/final_ab_new_users_upd_us.csv', 
                        dtype={'device': 'category', 'region': 'category'})
    user_events = pd.read_csv('/datasets/final_ab_events_upd_us.csv', parse_dates=['event_dt'])
    participants = pd.read_csv('/datasets/final_ab_participants_upd_us.csv')
    

In [4]:
# creating a function to print each column's unique values and their value counts

def exploration(data, parameter):
    print("Overview of '{}' column:".format(parameter))
    print()
    print(data[parameter].unique())
    print()
    print(data[parameter].value_counts())
    print()
    print('------------------------------------')
    print()
    

Now that I loaded the necessary libraries, saved the dataset in variables and created a function to assist in easy exploration of the datasets - I will perform an initial exploration of each dataset and then summarize my findings. 

### Marketing Events

In [5]:
# printing the "marketing_events" dataset info

marketing_events.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14 entries, 0 to 13
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   name       14 non-null     object
 1   regions    14 non-null     object
 2   start_dt   14 non-null     object
 3   finish_dt  14 non-null     object
dtypes: object(4)
memory usage: 576.0+ bytes


In [6]:
# printing the "marketing_events" dataset

marketing_events.sort_values(by="start_dt")


,name,regions,start_dt,finish_dt
6,Chinese New Year Promo,APAC,2020-01-25,2020-02-07
1,St. Valentine's Day Giveaway,"EU, CIS, APAC, N.America",2020-02-14,2020-02-16
8,International Women's Day Promo,"EU, CIS, APAC",2020-03-08,2020-03-10
2,St. Patric's Day Promo,"EU, N.America",2020-03-17,2020-03-19
3,Easter Promo,"EU, CIS, APAC, N.America",2020-04-12,2020-04-19
7,Labor day (May 1st) Ads Campaign,"EU, CIS, APAC",2020-05-01,2020-05-03
9,Victory Day CIS (May 9th) Event,CIS,2020-05-09,2020-05-11
11,Dragon Boat Festival Giveaway,APAC,2020-06-25,2020-07-01
4,4th of July Promo,N.America,2020-07-04,2020-07-11
13,Chinese Moon Festival,APAC,2020-10-01,2020-10-07


### User Events

In [7]:
# printing the "user_events" dataset info

user_events.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 423761 entries, 0 to 423760
Data columns (total 4 columns):
 #   Column      Non-Null Count   Dtype         
---  ------      --------------   -----         
 0   user_id     423761 non-null  object        
 1   event_dt    423761 non-null  datetime64[ns]
 2   event_name  423761 non-null  object        
 3   details     60314 non-null   float64       
dtypes: datetime64[ns](1), float64(1), object(2)
memory usage: 12.9+ MB


In [8]:
user_events['user_id'].count()-user_events['details'].isnull().count()

0

In [9]:
# printing the "user_events" dataset decription

user_events.describe(include='all').T


,count,unique,top,freq,first,last,mean,std,min,25%,50%,75%,max
user_id,423761,58703,A3917F81482141F2,36,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
event_dt,423761,257138,2020-12-14 18:54:55,10,2020-12-07 00:00:33,2020-12-30 23:36:33,NaN,NaN,NaN,NaN,NaN,NaN,NaN
event_name,423761,4,login,182465,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
details,60314.0,NaN,NaN,NaN,NaT,NaT,23.881219,72.228884,4.99,4.99,4.99,9.99,499.99


In [10]:
# printing a sample of the "user_events" dataset

user_events.head()


,user_id,event_dt,event_name,details
0,E1BDDCE0DAFA2679,2020-12-07 20:22:03,purchase,99.99
1,7B6452F081F49504,2020-12-07 09:22:53,purchase,9.99
2,9CD9F34546DF254C,2020-12-07 12:59:29,purchase,4.99
3,96F27A054B191457,2020-12-07 04:02:40,purchase,4.99
4,1FD7660FDF94CA1F,2020-12-07 10:15:09,purchase,4.99


In [11]:
# using a loop for implementing the "exploration" function on the "user_events" dataset's columns

columns = ["user_id", "event_dt", "event_name", "details"]

for parameter in columns:
    exploration(user_events, parameter)
    

Overview of 'user_id' column:

['E1BDDCE0DAFA2679' '7B6452F081F49504' '9CD9F34546DF254C' ...
 'C9F1399FDCD83F13' '4F32B58E4FF19DC5' '27767D4BECDD260E']

A3917F81482141F2    36
8AEB716C4CD82CA7    32
426330B820B8711A    32
BA87FD8D70D63CC6    32
11285A53EC2B2042    32
                    ..
DE755BE0D83CF729     1
5F55C1716703C6D0     1
756910FEB8E6E104     1
14A28942B782F678     1
031060788BD3E254     1
Name: user_id, Length: 58703, dtype: int64

------------------------------------

Overview of 'event_dt' column:

['2020-12-07T20:22:03.000000000' '2020-12-07T09:22:53.000000000'
 '2020-12-07T12:59:29.000000000' ... '2020-12-30T12:21:24.000000000'
 '2020-12-30T10:54:15.000000000' '2020-12-30T10:59:09.000000000']

2020-12-14 18:54:55    10
2020-12-23 02:37:24    10
2020-12-20 02:51:18     9
2020-12-24 00:54:52     9
2020-12-21 21:14:13     9
                       ..
2020-12-18 20:58:34     1
2020-12-18 12:40:24     1
2020-12-18 15:18:40     1
2020-12-18 05:01:00     1
2020-12-30 10:59:09

### Users

In [12]:
# printing the "users" dataset info

users.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 58703 entries, 0 to 58702
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   user_id     58703 non-null  object  
 1   first_date  58703 non-null  object  
 2   region      58703 non-null  category
 3   device      58703 non-null  category
dtypes: category(2), object(2)
memory usage: 1.0+ MB


In [13]:
# printing the "users" dataset decription

users.describe(include='all').T


,count,unique,top,freq
user_id,58703,58703,D72A72121175D8BE,1
first_date,58703,17,2020-12-21,6077
region,58703,4,EU,43396
device,58703,4,Android,26159


In [14]:
# printing a sample of the "users" dataset

users.head()


,user_id,first_date,region,device
0,D72A72121175D8BE,2020-12-07,EU,PC
1,F1C668619DFE6E65,2020-12-07,N.America,Android
2,2E1BF1D4C37EA01F,2020-12-07,EU,PC
3,50734A22C0C63768,2020-12-07,EU,iPhone
4,E1BDDCE0DAFA2679,2020-12-07,N.America,iPhone


In [15]:
# using a loop for implementing the "exploration" function on the "users" dataset's columns

columns = ["user_id", "first_date", "region", "device"]

for parameter in columns:
    exploration(users, parameter)


Overview of 'user_id' column:

['D72A72121175D8BE' 'F1C668619DFE6E65' '2E1BF1D4C37EA01F' ...
 '7ADEE837D5D8CBBD' '1C7D23927835213F' '8F04273BB2860229']

D72A72121175D8BE    1
87818F11DF867086    1
54053B51909746FA    1
6286DB850DAC4A39    1
39D8D335E253C151    1
                   ..
9212B9208445C44A    1
CD1F82589F1EA3F7    1
827B6C860070DF84    1
DA6EB5BBD34FFC82    1
8F04273BB2860229    1
Name: user_id, Length: 58703, dtype: int64

------------------------------------

Overview of 'first_date' column:

['2020-12-07' '2020-12-14' '2020-12-21' '2020-12-08' '2020-12-15'
 '2020-12-22' '2020-12-09' '2020-12-16' '2020-12-23' '2020-12-10'
 '2020-12-17' '2020-12-11' '2020-12-18' '2020-12-12' '2020-12-19'
 '2020-12-13' '2020-12-20']

2020-12-21    6077
2020-12-14    5448
2020-12-07    5291
2020-12-13    4181
2020-12-20    4140
2020-12-12    3591
2020-12-19    3480
2020-12-18    3238
2020-12-22    3083
2020-12-08    3017
2020-12-17    2940
2020-12-15    2924
2020-12-10    2784
2020-12-11    2

### Participants

In [16]:
# printing the "participants" dataset info

participants.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14525 entries, 0 to 14524
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   user_id  14525 non-null  object
 1   group    14525 non-null  object
 2   ab_test  14525 non-null  object
dtypes: object(3)
memory usage: 340.6+ KB


In [17]:
# printing the "participants" dataset decription

participants.describe(include='all').T


,count,unique,top,freq
user_id,14525,13638,307C06C0E6F09B93,2
group,14525,2,A,8214
ab_test,14525,2,interface_eu_test,10850


In [18]:
# printing a sample of the "participants" dataset

participants.head()


,user_id,group,ab_test
0,D1ABA3E2887B6A73,A,recommender_system_test
1,A7A3664BD6242119,A,recommender_system_test
2,DABC14FDDFADD29E,A,recommender_system_test
3,04988C5DF189632E,A,recommender_system_test
4,4FF2998A348C484F,A,recommender_system_test


In [19]:
# using a loop for implementing the "exploration" function on the "participants" dataset's columns

columns = ["user_id", "group", "ab_test"]

for parameter in columns:
    exploration(participants, parameter)
    

Overview of 'user_id' column:

['D1ABA3E2887B6A73' 'A7A3664BD6242119' 'DABC14FDDFADD29E' ...
 'F501F79D332BE86C' '63FBE257B05F2245' '79F9ABFB029CF724']

307C06C0E6F09B93    2
74C5CCDA7DF761EE    2
6445624A3AE163E6    2
3A6D88AB973BD99D    2
C1E43A9B1D675927    2
                   ..
B39C1D7760CF809C    1
4E6C94A7969F5079    1
7EEEA8AA76EC8703    1
9EC259921F8521B1    1
79F9ABFB029CF724    1
Name: user_id, Length: 13638, dtype: int64

------------------------------------

Overview of 'group' column:

['A' 'B']

A    8214
B    6311
Name: group, dtype: int64

------------------------------------

Overview of 'ab_test' column:

['recommender_system_test' 'interface_eu_test']

interface_eu_test          10850
recommender_system_test     3675
Name: ab_test, dtype: int64

------------------------------------



### Summary

1. marketing_events
    * The dataset has 4 columns and 14 rows;
    * Most events are for specific regions, about 3 are for all regions. 
    
    
2. user_events
    * The dataset has 4 columns and 423,761 rows;
    * The "details" column has only 60,314 non null values - the nature of the null values will be examined in the next section;
    * There are 58,703 users and 257,138 event dates and 4 types of events (the most common are "login" and "product_page";
    * There are only 4 prices: 4.99, 9.99, 99.99 and 499.99. 4.99 is by far the most frequent. 


3. users
    * The dataset has 4 columns and 58,703 rows (matches the number of users in the "user_events" dataset);
    * The most common region and sign up device are EU and Android (then PC), respectively;
    * Most users signed up on December 7th, the 14th and the 21st, 2020;
    * There are users who signed up after the exclusion date (December 21st).


4. participants
    * The dataset has 3 columns and 14,525 rows;
    * There are only 13,638 participants in the dataset (meaning there are a lot of users who aren't participants);
    * 3/4 of the participants (10,850 to be exact) participated in the "interface_eu_test" (not the test analyzed in this report);
    * This dataset is an early indication that the test was not performed as intended: 3,675 participants instead of 6,000 and it seems that some might be participants of the other test as well. 

## Data Preprocessing

In this section I will perform data preprocessing - convert data types, explore duplicates and null values and more. 

In [20]:
# creating a function to print the number of duplicate and null values in each column of a selected dataset

def dupes_and_nulls(data):
    for i in data[data.duplicated()].columns:
        print('Number of duplicate values in "{}" column: {}'.format(i, 
                        data[data.duplicated()][i].nunique()))
    print('----------------------')
    for i in data:
        if data[i].isnull().sum()>0:
            print('Number of null values in "{}" column: {}'.format(i, data[i].isnull().sum()))
        else:
            print('No null values in "{}" column'.format(i))
            

### Marketing Events

In [21]:
# converting the "start_dt" and "finish_dt" columns to datetime and checking the implementation by printing info

marketing_events['start_dt'] = marketing_events['start_dt'].map(lambda x: dt.datetime.strptime(x, '%Y-%m-%d'))
marketing_events['finish_dt'] = marketing_events['finish_dt'].map(lambda x: dt.datetime.strptime(x, '%Y-%m-%d'))
marketing_events.info('deep')


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14 entries, 0 to 13
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   name       14 non-null     object        
 1   regions    14 non-null     object        
 2   start_dt   14 non-null     datetime64[ns]
 3   finish_dt  14 non-null     datetime64[ns]
dtypes: datetime64[ns](2), object(2)
memory usage: 576.0+ bytes


In the marketing_events dataset, little preprocessing was required - only converting the data types of two columns to datetime. 

### User Events

In [22]:
# checking whether there's need to alter datatypes in the "user_events" dataset by printing info

user_events.info('deep')


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 423761 entries, 0 to 423760
Data columns (total 4 columns):
 #   Column      Non-Null Count   Dtype         
---  ------      --------------   -----         
 0   user_id     423761 non-null  object        
 1   event_dt    423761 non-null  datetime64[ns]
 2   event_name  423761 non-null  object        
 3   details     60314 non-null   float64       
dtypes: datetime64[ns](1), float64(1), object(2)
memory usage: 12.9+ MB


In this dataset, there is no need to convert data types. However, the null values are significant and their nature needs to be explored. 

In [23]:
# implementing the "dupes_and_nulls" function on the "user_events" dataset's columns

dupes_and_nulls(user_events)


Number of duplicate values in "user_id" column: 0
Number of duplicate values in "event_dt" column: 0
Number of duplicate values in "event_name" column: 0
Number of duplicate values in "details" column: 0
----------------------
No null values in "user_id" column
No null values in "event_dt" column
No null values in "event_name" column
Number of null values in "details" column: 363447


In [24]:
# creating a table with only the null values in the "details" column of the "user_events" dataset

null_details = user_events[(user_events.details.isnull())]


In [25]:
# checking the percentage of null values 

print("""The percent of null values in the 'details' column is {:.2f}%.""".format
      (len(null_details)/len(user_events)*100))


The percent of null values in the 'details' column is 85.77%.


In [26]:
# checking whether there are null values in all of the events, including "purchase"

null_details.groupby('event_name').count()


,user_id,event_dt,details
event_name,,,
login,182465,182465,0
product_cart,60120,60120,0
product_page,120862,120862,0


Even though more than 85% of the values in the "details" column are null - there is a reasonable explanation for this and there is no need to remove them or fill them. The details is the priced paid for products users purchased and there are no null values for the "purchase" event. 

In [27]:
# locating the earlier and latest dates in the "user_events" dataset

print("""The earliest date in the 'user_events' dataset is: {}
The latest date in the 'user_events' dataset is: {}""".format(user_events.event_dt.min(), user_events.event_dt.max()))


The earliest date in the 'user_events' dataset is: 2020-12-07 00:00:33
The latest date in the 'user_events' dataset is: 2020-12-30 23:36:33


The dates of the test are from December 7th, 2020, till January 1st, 2021 - so there is no need to remove events since they all fall within these dates. 

### Users

In [28]:
# converting the "first_date" column to datetime and checking the implementation by printing info

users['first_date'] = users['first_date'].map(lambda x: dt.datetime.strptime(x, '%Y-%m-%d'))
users.info('deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 58703 entries, 0 to 58702
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   user_id     58703 non-null  object        
 1   first_date  58703 non-null  datetime64[ns]
 2   region      58703 non-null  category      
 3   device      58703 non-null  category      
dtypes: category(2), datetime64[ns](1), object(1)
memory usage: 1.0+ MB


In [29]:
# locating the earlier and latest dates in the "users" dataset

print("""The earliest date in the 'users' dataset is: {}
The latest date in the 'users' dataset is: {}""".format(users.first_date.min(), users.first_date.max()))

The earliest date in the 'users' dataset is: 2020-12-07 00:00:00
The latest date in the 'users' dataset is: 2020-12-23 00:00:00


In [30]:
# implementing the "dupes_and_nulls" function on the "users" dataset's columns

dupes_and_nulls(users)


Number of duplicate values in "user_id" column: 0
Number of duplicate values in "first_date" column: 0
Number of duplicate values in "region" column: 0
Number of duplicate values in "device" column: 0
----------------------
No null values in "user_id" column
No null values in "first_date" column
No null values in "region" column
No null values in "device" column


In this dataset, there was an incorrect data type for the "first_date" column and there are users who signed up after December 21st, 2020. 

### Participants

In [31]:
# implementing the "dupes_and_nulls" function on the "participants" dataset's columns

dupes_and_nulls(participants)


Number of duplicate values in "user_id" column: 0
Number of duplicate values in "group" column: 0
Number of duplicate values in "ab_test" column: 0
----------------------
No null values in "user_id" column
No null values in "group" column
No null values in "ab_test" column


### Checking Registration Date and Filtering Data

In [32]:
# creating a table of users who registered after December 21st, 2020

past_date = users.query('first_date > "2020-12-21"')
past_date # printing the "past_date" table


,user_id,first_date,region,device
22757,5815F7ECE74D949F,2020-12-22,CIS,PC
22758,32EAEA5E903E3BC1,2020-12-22,N.America,Android
22759,9DF7A3C46487EF0B,2020-12-22,EU,Android
22760,ADE98C6440423287,2020-12-22,EU,iPhone
22761,5A5833D3AEA75255,2020-12-22,N.America,PC
...,...,...,...,...
32118,165AFCBF42C043F8,2020-12-23,EU,PC
32119,54E7F36C0E976E24,2020-12-23,EU,Android
32120,7E43EB2E03A33E78,2020-12-23,EU,PC
32121,B8B679DEE9F2CA06,2020-12-23,EU,PC


#### Filtering the Users Dataset

In [33]:
# creating a new "users" dataset without users who signed up after December 21st, 2020

filtered_users = users[~users.user_id.isin(past_date['user_id'])]
filtered_users.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 53440 entries, 0 to 58702
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   user_id     53440 non-null  object        
 1   first_date  53440 non-null  datetime64[ns]
 2   region      53440 non-null  category      
 3   device      53440 non-null  category      
dtypes: category(2), datetime64[ns](1), object(1)
memory usage: 1.3+ MB


In [34]:
# checking how many users signed up on each date 

filtered_users.groupby('first_date')['user_id'].count().reset_index()


,first_date,user_id
0,2020-12-07,5291
1,2020-12-08,3017
2,2020-12-09,2010
3,2020-12-10,2784
4,2020-12-11,2226
5,2020-12-12,3591
6,2020-12-13,4181
7,2020-12-14,5448
8,2020-12-15,2924
9,2020-12-16,2093


In [ ]:
# checking that filtering didn't distort the distribution 

fig = make_subplots(rows=2, cols=2, shared_yaxes=True, subplot_titles=
                    ("Regional Distribution","Device Distribution"))

fig.add_trace(go.Histogram(x=users['region'], y=users['user_id'], name="Not Filtered",
                    marker=dict(color=[px.colors.sequential.Pinkyl_r], coloraxis="coloraxis")), 1, 1)

fig.add_trace(go.Histogram(x=filtered_users['region'], y=filtered_users['user_id'],name="Filtered",
                    marker=dict(color=[px.colors.sequential.Pinkyl_r], coloraxis="coloraxis")),
              2, 1)

fig.add_trace(go.Histogram(x=users['device'], y=users['user_id'],name="Not Filtered",
                    marker=dict(color=[px.colors.sequential.Pinkyl_r], coloraxis="coloraxis")),
              1, 2)

fig.add_trace(go.Histogram(x=filtered_users['device'], y=filtered_users['user_id'],name="Filtered",
                    marker=dict(color=[px.colors.sequential.Pinkyl_r], coloraxis="coloraxis")),
              2, 2)

fig.update_traces(marker=dict(color=(px.colors.sequential.Teal)))
fig.update_layout(showlegend=False, title_text="Distribution Before and After Filteration", yaxis=dict(
        title='Number of Users'))
fig.show()


#### Filtering the User Events Dataset

In [35]:
# creating a new "user_events" dataset without users who signed up after December 21st, 2020

filtered_user_events = user_events[~user_events.user_id.isin(past_date['user_id'])]
filtered_user_events.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 394657 entries, 0 to 423760
Data columns (total 4 columns):
 #   Column      Non-Null Count   Dtype         
---  ------      --------------   -----         
 0   user_id     394657 non-null  object        
 1   event_dt    394657 non-null  datetime64[ns]
 2   event_name  394657 non-null  object        
 3   details     56170 non-null   float64       
dtypes: datetime64[ns](1), float64(1), object(2)
memory usage: 15.1+ MB


In [ ]:
# checking that filtering didn't distort the distribution 

fig = make_subplots(rows=1, cols=2, shared_yaxes=True, subplot_titles=
                    ("Original Distribution","Filtered Distribution"))

fig.add_trace(go.Histogram(x=user_events['event_name'], y=user_events['user_id'], name="Not Filtered",
                    marker=dict(coloraxis="coloraxis")), 1, 1)

fig.add_trace(go.Histogram(x=filtered_user_events['event_name'], y=filtered_user_events['user_id'],name="Filtered",
                    marker=dict(coloraxis="coloraxis")),
              1, 2)

fig.update_traces(marker=dict(color=(px.colors.sequential.Teal_r)))
fig.update_layout(showlegend=False, title_text="Distribution Before and After Filteration", yaxis=dict(
        title='Number of Users'))
fig.show()


#### Filtering the Participants Dataset

In [36]:
# creating a new "participants" dataset without users who signed up after December 21st, 2020

filtered_participants = participants[~participants.user_id.isin(past_date['user_id'])]
filtered_participants.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 13523 entries, 0 to 14524
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   user_id  13523 non-null  object
 1   group    13523 non-null  object
 2   ab_test  13523 non-null  object
dtypes: object(3)
memory usage: 422.6+ KB


In [ ]:
# checking that filtering didn't distort the distribution 

fig = make_subplots(rows=2, cols=2, shared_yaxes=True, subplot_titles=
                    ("Original Distribution","Filtered Distribution"))

fig.add_trace(go.Histogram(x=participants['group'], y=participants['user_id'], name="Not Filtered",
                    marker=dict(coloraxis="coloraxis")), 1, 1)

fig.add_trace(go.Histogram(x=filtered_participants['group'], y=filtered_participants['user_id'],name="Filtered",
                    marker=dict(coloraxis="coloraxis")),
              1, 2)

fig.add_trace(go.Histogram(x=participants['ab_test'], y=participants['user_id'], name="Not Filtered",
                    marker=dict(coloraxis="coloraxis")), 2, 1)

fig.add_trace(go.Histogram(x=filtered_participants['ab_test'], y=filtered_participants['user_id'],name="Filtered",
                    marker=dict(coloraxis="coloraxis")),
              2, 2)

fig.update_traces(marker=dict(color=(px.colors.sequential.Teal)))
fig.update_layout(showlegend=False, title_text="Distribution Before and After Filteration", yaxis=dict(
        title='Number of Users'))
fig.show()


### Summary

* In the marketing_events dataset I converted the data types of two columns to datetime.
* In the user_events dataset: 
    * Even though more than 85% of the values in the "details" column are null - there is a reasonable explanation for this and there is no need to remove them or fill them. The details is the priced paid for products users purchased and there are no null values for the "purchase" event;
    * The dates of the test are from December 7th, 2020, till January 1st, 2021 - so there is no need to remove events from the dataset since they all fall within these dates.
* In the users dataset there was an incorrect data type for the "first_date" column and there are users who signed up after December 21st, 2020.
* In the participants dataset there were no duplicates or nulls and no data types to convert. 

- Sign up date filtration 
    * Since there were users who signed up after December 21st, 2020, in the datasets - I saved their user ids in a table called "past_date" and filtered their ids from the users, user_events and participants datasets;
    * I checked that the filtration did not create anomalies in the distribution of values in the datasets - the distribution stayed the same only in fewer numbers;
    * Once these users were filtered out, the rows of each data set were as follows: 
        * users - 53,440;
        * user_events - 394,657 (56,170 non null values in the "details" column);
        * participants - 13,523.

## Exploratory Data Analysis

Now that I concluded the data preprocessing section, in the following section I will perform the exploratory data analysis.

### Funnel Stages - Conversion 

In [ ]:
# creating an event funnel for all users in the "user_events" dataset

# creating a table with the number of unique user per event and reindexing the events in the table 
funnel = filtered_user_events.groupby('event_name')['user_id'].nunique().reset_index().reindex(
    [0, 2, 1, 3]).reset_index(drop=True)

fig = go.Figure(go.Funnel(
    y = funnel['event_name'],
    x = funnel['user_id'],
    textinfo = "value + percent previous + percent initial"))
fig.update_traces(marker=dict(color=(px.colors.sequential.Teal_r
                                    )))
fig.update_layout(
    title_text="Overall Filtered Data Event Funnel", width=1100, height=570,plot_bgcolor='white', 
    paper_bgcolor='white')
fig.show()


Overall, we can see that more than 60% of users who login to the store, visit a product's page and nearly 50% purchase the/a product. However, we can also see that it's possible to purchase a product directly from product page and skip the product cart since the purchase event has more users than the product cart event. 

In [ ]:
# checking the number of participants of the "recommender_system_test" per event 

# creating a new dataset by merging the "participants" dataset to the "user_events" dataset
merged_user_events = filtered_user_events.merge(filtered_participants, on='user_id', how='left')

# slicing the merged dataset to include on the test and reindexing the events in the table
test_funnel = merged_user_events.query('ab_test == "recommender_system_test"').groupby(
    ['event_name', 'group'])['user_id'].nunique().reset_index().reindex(
    [0, 1, 4, 5, 2, 3, 6, 7]).reset_index(drop=True) 

fig = px.bar(test_funnel, x="event_name", 
                   y="user_id", color= 'group', title='Distribution of Users Across Regions', 
                   barmode="group", color_discrete_sequence=(px.colors.sequential.Teal_r))
fig.update_layout(bargap=0.2, 
    xaxis_tickfont_size=14,
    yaxis=dict(
        title='Number of Users',
        titlefont_size=16,
        tickfont_size=14), xaxis=dict(
        title="Event",
        titlefont_size=16,
        tickfont_size=14))
fig.show()


As we can see from the distribution of participants of the "recommender_system_test" per group - most users logged in and view the product page and approximately the same amount of users visited the cart/made a purchase. It should also be noted that the groups are grossly uneven. 

In [ ]:
# creating an event funnel for the participants of the "recommender_system_test" per test group

fig = go.Figure()

fig.add_trace(go.Funnel(
    name = 'A',
    y = test_funnel.query("group == 'A'")['event_name'],
    x = test_funnel.query("group == 'A'")['user_id'],
    textinfo = "value + percent previous + percent initial"))

fig.add_trace(go.Funnel(
    name = 'B',
    y = test_funnel.query("group == 'B'")['event_name'],
    x = test_funnel.query("group == 'B'")['user_id'],
    textinfo = "value + percent previous + percent initial"))

fig.update_traces(marker=dict(color=(px.colors.sequential.Teal)))
fig.update_layout(
    title_text="Filtered Data Event Funnel per Experiment Group", width=930, height=570, 
    plot_bgcolor='white', paper_bgcolor='white')

fig.show()


For the events performed by the participants of the "recommender_system_test", we can see that group A is significantly smaller than group B and that for both groups the trends of the overall users are the same. 

### Distribution of Number of Events per User per Sample

In [ ]:
# checking the distribution of events per user 

events_dis = filtered_user_events.groupby('user_id')['event_name'].count().reset_index().sort_values(by=
            'event_name', ascending = False) # creating a table with the number of events per user

# checking the distribution by grouping by the number of users and counting how many users had n_events
fig = px.histogram(events_dis.groupby('event_name')['user_id'].count().reset_index().rename(columns=
                                  {'event_name': 'n_events', 'user_id':'n_users'}), x="n_events", 
                   y="n_users", title='Distribution of Number of Events per User', 
                   nbins=50, color_discrete_sequence=(px.colors.sequential.Teal_r))
fig.update_layout(bargap=0, 
    xaxis_tickfont_size=14,
    yaxis=dict(
        title='Number of Users',
        titlefont_size=16,
        tickfont_size=14), xaxis=dict(
        title="Number of Events",
        titlefont_size=16,
        tickfont_size=14))
fig.show()


Most users in the dataset had either 4, 6, 8 or 12 events between December 7th, 2020, and January 1st, 2021 (actual last date is December 30th, 2020). Very few users had more than 12 events. 

In [ ]:
# creating a new dataset by merging the "filtered_participants" dataset to the "filtered_user_events" dataset

merged_events = filtered_user_events.merge(filtered_participants, on='user_id', how='left')

# creating a table with the number of events per participants of the "recommender_system_test" per test group 

rst = merged_events.groupby(['user_id', 'ab_test', 'group'])[
    'event_name'].count().reset_index().query("ab_test == 'recommender_system_test'") 

# checking the distribution by grouping by the number of participants and counting how many users had n_events

rst_e = rst.groupby(['event_name', 'group'])['user_id'].count().reset_index().rename(columns=
                                  {'event_name': 'n_events', 'user_id':'n_users'})
fig = px.histogram(rst_e, x="n_events", 
                   y="n_users", color= 'group', title='Distribution of Number of Events per User - recommender_system_test', 
                   nbins=50, color_discrete_sequence=(px.colors.sequential.Teal_r))
fig.update_layout(bargap=0, 
    xaxis_tickfont_size=14,
    yaxis=dict(
        title='Number of Users',
        titlefont_size=16,
        tickfont_size=14), xaxis=dict(
        title="Number of Events",
        titlefont_size=16,
        tickfont_size=14))
fig.show()


The participants of the "recommender_system_test" follow the same trends of all the users, with more than 800 members having 6 events. Overall, most users had no more than 12 events. 

### Distribution of Number of Events per Day in Test Period

In [ ]:
# checking the distribution of events per date per test 

merged_user_events = filtered_user_events.merge(filtered_participants, on='user_id', how='left').fillna('none')
merged_user_events['date'] = merged_user_events['event_dt'].astype('datetime64[D]')

fig = px.histogram(merged_user_events, x="date", color='ab_test', title='Histogram of Events by Date', 
                   nbins=30, color_discrete_sequence=(px.colors.sequential.Teal_r))
fig.update_layout(bargap=0.2, 
    xaxis_tickfont_size=12,
    yaxis=dict(
        title='Number of Events',
        titlefont_size=14,
        tickfont_size=12), xaxis=dict(
        title="Event Date",
        titlefont_size=14,
        tickfont_size=12))
fig.show()


December 21st, 2020, had the most events between December 7th and December 30th. There was a steady climb in events per day up to December 14th and then decline, a peak on the 21st and then a decline again till Christmas Eve (with no events on Christmas) to New Year's Eve and the New Year. 

In [ ]:
# checking the distribution of events per date per experiment group in the "recommender_system_test"

fig = px.histogram(merged_user_events.query("ab_test == 'recommender_system_test'"), x="date", color='group', 
                   title='Histogram of Events by Date - recommender_system_test', 
                   nbins=30, color_discrete_sequence=(px.colors.sequential.Teal_r))
fig.update_layout(bargap=0.2, 
    xaxis_tickfont_size=14,
    yaxis=dict(
        title='Number of Events',
        titlefont_size=14,
        tickfont_size=12), xaxis=dict(
        title="Event Date",
        titlefont_size=14,
        tickfont_size=12))
fig.show()

The distribution of events per day among the participants of the "recommender_system_test" is similar to the overall distribution, with one exception - no mini peak of December 24th. 

### Overlapping Users in Test Groups

#### Locating Overlapping Users

##### Two Test Groups

In [37]:
# locating participants who are members of both A and B group 

two_grouped = filtered_participants.groupby(['user_id']).agg({'group': 'nunique'}).sort_values(by='group', 
                                                ascending=False).query('group > 1').reset_index()
two_grouped


,user_id,group
0,BEF16764A13AEC34,2
1,C89E24F4C723CEF4,2
2,B582D211C1BE1FFD,2
3,F507CB8030C885FB,2
4,3A6D88AB973BD99D,2
...,...,...
436,C474F67E2204321C,2
437,83E9CFC64D61108E,2
438,BF6D4EF4B54B3E94,2
439,7C9CA3F88B8755F1,2


As we can see above, there are participants who are members of both groups - but not necessarily in the same test. 

##### Two Test Groups in One ab_test

In [38]:
# checking if there are participants who are members of more than group in one test

two_grouped_test = filtered_participants.groupby(['user_id']).agg({'group': 'nunique', 'ab_test': 'nunique'}).sort_values(by='group', 
                                                ascending=False).query('group > 1 & ab_test == 1').reset_index()
two_grouped_test


,user_id,group,ab_test


As we can, there are no participants who are members of both groups in either of the tests in the dataset. 

##### Two ab_tests

In [39]:
# checking if there are participants who are members of more than one test

two_tested = participants.groupby(['user_id']).agg({'ab_test': 'nunique'}).sort_values(by='ab_test', 
                                                ascending=False).query('ab_test > 1').reset_index()
two_tested.head()

,user_id,ab_test
0,83AB3C13B3972BAE,2
1,E0EEE04D9FDB4931,2
2,6786C23A54D59DD4,2
3,21C2D41E1FDF1E70,2
4,21AEB217BA764EAB,2


In [40]:
print("""There {} participants who are members of both the "recommender_system_test" and the "interface_eu_test". Once 
removed, there will be {} "recommender_system_test" participants remaining.""".format(two_tested['user_id'].nunique(), 
                                        filtered_participants.query("ab_test == 'recommender_system_test'")
                                        ['user_id'].nunique() - two_tested['user_id'].nunique())) 


There 887 participants who are members of both the "recommender_system_test" and the "interface_eu_test". Once 
removed, there will be 2788 "recommender_system_test" participants remaining.


#### Filtering the Participants, Users ad User Event Datasets

##### Filtering Two Tested Participants

In [41]:
# filtering from the "filtered_participants" dataset the participants who are members of both tests

filtered_participants = filtered_participants[~filtered_participants.user_id.isin(two_tested['user_id'])]


In [42]:
# rechecking whether there are still participants who are members of both tests

filtered_participants.groupby(['user_id']).agg({'ab_test': 'nunique'}).sort_values(by='ab_test', 
                                                ascending=False).query('ab_test > 1').reset_index()


,user_id,ab_test


In [43]:
# filtering from the "filtered_users" dataset the participants who are members of both tests

filtered_users = filtered_users[~filtered_users.user_id.isin(two_tested['user_id'])]


In [44]:
# filtering from the "filtered_user_events" dataset the participants who are members of both tests

filtered_user_events = filtered_user_events[~filtered_user_events.user_id.isin(two_tested['user_id'])]


##### Filtering None "recommender_system_test" Participants

In [45]:
# creating a dataset with only "recommender_system_test" participants, since the "participants" dataset contains members of two test 

rst_participants = filtered_participants.query("ab_test == 'recommender_system_test'")


In [46]:
# creating a filtered "users" dataset with only the "recommender_system_test" participants

rst_users = filtered_users[filtered_users.user_id.isin(rst_participants['user_id'])]


In [47]:
# creating a filtered "user_events" dataset with only the "recommender_system_test" participants

rst_events = filtered_user_events[filtered_user_events.user_id.isin(rst_participants['user_id'])]


#### Evaluating the Amount of Data Removed while Filtering

In [ ]:
# checking how was much of the data was filtered out by keeping only participants of "recommender_system_test"

labels = ["recommender_system_test", "others"] # setting labels for slices 

fig = make_subplots(rows=1, cols=2, specs=[[{'type':'domain'}, {'type':'domain'}]]) # creating subplots

# creating donut no.1 of number of unique users remaining and filtered out
fig.add_trace(go.Pie(labels=labels, values=[rst_events['user_id'].nunique(), user_events
                            ['user_id'].nunique()-rst_events['user_id'].nunique()], name="Users"),1, 1)

# creating donut no.2 of number of events remaining and filtered out
fig.add_trace(go.Pie(labels=labels, values=[rst_events['event_name'].count(), user_events
                            ['event_name'].count()-rst_events['event_name'].count()], name="Events"),1, 2)

fig.update_traces(pull=[0, 0.2], hole=.3, hoverinfo="label+value+percent+name", marker=dict
                  (colors=(px.colors.sequential.Teal))) # pulling out a slice, setting hole, color and hover info

# setting the chart's title and the text in the donut holes 
fig.update_layout(
    title_text="Percentage of Data Removed While Filtering",
    annotations=[dict(text='Users', x=0.185, y=0.425, font_size=20, showarrow=False),
                 dict(text='Events', x=0.827, y=0.426, font_size=20, showarrow=False)])

fig.show()


I have created new tables of each dataset with only the data pertaining to the participants of the "recommender_system_test" who participated in this test alone. The data filtered out is more than 95% of users and events. 

### Additional Details

Now that I created clean datasets with only the "recommender_system_test" participants, I will merge these datasets to one easy to work with data set and examine distribution of participants' region and device used signing up. 

In [48]:
# creating a merged dataset containing all the info pertaining only to the participants of the "recommender_system_test"

rst_merge = rst_events.merge(rst_participants, on='user_id', how='left')
rst_merge = rst_merge.merge(rst_users, on='user_id', how='left')
rst_merge.head()


,user_id,event_dt,event_name,details,group,ab_test,first_date,region,device
0,831887FE7F2D6CBA,2020-12-07 06:50:29,purchase,4.99,A,recommender_system_test,2020-12-07,EU,Android
1,3C5DD0288AC4FE23,2020-12-07 19:42:40,purchase,4.99,A,recommender_system_test,2020-12-07,EU,PC
2,49EA242586C87836,2020-12-07 06:31:24,purchase,99.99,B,recommender_system_test,2020-12-07,EU,iPhone
3,A640F31CAC7823A6,2020-12-07 18:48:26,purchase,4.99,B,recommender_system_test,2020-12-07,EU,PC
4,A9908F62C41613A8,2020-12-07 11:26:47,purchase,9.99,B,recommender_system_test,2020-12-07,EU,PC


#### Regional Distribution

In [ ]:
# checking the distribution of region among the participants of the "recommender_system_test" per group

fig = px.histogram(rst_merge.groupby(['region', 'group'])['user_id'].nunique().reset_index()
                   .sort_values(by='user_id', ascending=False), x="region", 
                   y="user_id", color= 'group', title='Distribution of Users Across Regions', 
                   nbins=50, color_discrete_sequence=(px.colors.sequential.Teal_r))
fig.update_layout(bargap=0.2, 
    xaxis_tickfont_size=14,
    yaxis=dict(
        title='Number of Users',
        titlefont_size=16,
        tickfont_size=14), xaxis=dict(
        title="Region",
        titlefont_size=16,
        tickfont_size=14))
fig.show()


Even though the intended audience was supposed to be from EU only, there are some participants from other regions among the participants. 

#### Distribution of Device Used When Signing Up

In [ ]:
# checking the distribution of the device the participants of the "recommender_system_test" signed up with in the test period

fig = px.histogram(rst_merge.groupby(['device', 'group', 'first_date'])['user_id'].count().
                   reset_index(), x="first_date", 
                   y="user_id", color= 'device', title='Distribution of Devices Used When Signing Up', 
                   nbins=50, color_discrete_sequence=(px.colors.sequential.Teal))
fig.update_layout(bargap=0.2, 
    xaxis_tickfont_size=14,
    yaxis=dict(
        title='Number of Users',
        titlefont_size=16,
        tickfont_size=14), xaxis=dict(
        title="Date of Signing Up",
        titlefont_size=16,
        tickfont_size=14))
fig.show()


Most participants signed up on December 7th, the 14th and the 21st. We can also see better sign up numbers leading to the 21st. Participants used mostly Android and PC to sign up. 

In [ ]:
# checking the distribution of the device the participants of the "recommender_system_test" signed up with per group

fig = px.histogram(rst_merge.groupby(['device', 'group', 'first_date'])['user_id'].nunique().
                   reset_index().sort_values(by='user_id', ascending=False), x="device", 
                   y="user_id", color= 'group', title='Distribution of Users Across Devices', 
                   nbins=50, color_discrete_sequence=(px.colors.sequential.Teal_r))
fig.update_layout(bargap=0.2, 
    xaxis_tickfont_size=14,
    yaxis=dict(
        title='Number of Users',
        titlefont_size=16,
        tickfont_size=14), xaxis=dict(
        title="Device Used While Signing Up",
        titlefont_size=16,
        tickfont_size=14))
fig.show()


Android is by far the device used the most to sign up - more than 1,200 participants (out of 2,788 participants). Next is PC and iPhone while Mac is last. It is possible that most participants sign up "on the go" maybe they saw an advertisement, or that the interface is more user friendly. 

### Summary

- Conversion
    * Overall, more than 60% of users who login to the store, visit a product's page and nearly 50% purchase the/a product. However, we can also see that it's possible to purchase a product directly from product page and skip the product cart since the purchase event has more users than the product cart event;
    * Most users logged in and view the product page and approximately the same amount of users visited the cart/made a purchase. It should also be noted that the groups are grossly uneven;
    * For the events performed by the participants of the "recommender_system_test", we can see that group A is significantly smaller than group B and that for both groups the trends of the overall users are the same. 
    
    
- Events per User
    * Most users in the dataset had either 4, 6, 8 or 12 events between December 7th, 2020, and January 1st, 2021 (actual last date is December 30th, 2020). Very few users had more than 12 events;
    * The participants of the "recommender_system_test" follow the same trends of all the users, with more than 800 members having 6 events. Overall, most users had no more than 12 events. 
    
    
- Events per Day
    * The participants of the "recommender_system_test" follow the same trends of all the users, with more than 800 members having 6 events. Overall, most users had no more than 12 events; 
    * The distribution of events per day among the participants of the "recommender_system_test" is similar to the overall distribution, with one exception - no mini peak of December 24th. 
    
- Overlapping test participants and filtering non-recommender_system_test participants
    * There were no participants who are members of both groups in either of the tests in the dataset;
    * There 887 participants who are members of both the "recommender_system_test" and the "interface_eu_test";
    * created new tables of each dataset with only the data pertaining to the participants of the "recommender_system_test" who participated in this test alone. The data filtered out is more than 95% of users and events. 

- Additional Details
    * I merged all the data pertaining to the participants of the "recommender_system_test" into one dataset;
    * Most of the participants are from the EU but there are some participants from other regions among the participants;
    * Most participants signed up on December 7th, the 14th and the 21st. There were better sign up numbers leading to the 21st. Participants used mostly Android and PC to sign up;
    * Android is by far the device used the most to sign up - more than 1,200 participants (out of 2,788 participants). Next is PC and iPhone while Mac is last. It is possible that most participants sign up "on the go" maybe they saw an advertisement, or that the interface is more user friendly.

## A/B Test Analysis

In this section I will examine the results of the "recommender_system_test" and perform a z-test to determine whether there is a statistical significance difference between the proportions of the groups at each event. 

### The A/B Test Results

#### Number of Participants and the Distribution of Participants per Test Group

In [ ]:
# checking the number of users in each group of the "recommender_system_test"

fig = px.histogram(rst_merge.groupby('group')['user_id'].nunique().reset_index(), x="group", 
                   y="user_id", title='Distribution of Participants per Test Group', 
                   nbins=50, color_discrete_sequence=(px.colors.sequential.Darkmint_r, 4))
fig.update_layout(bargap=0.1, 
    xaxis_tickfont_size=14,
    yaxis=dict(
        title='Number of Participants',
        titlefont_size=16,
        tickfont_size=14), xaxis=dict(
        title="Test Groups",
        titlefont_size=16,
        tickfont_size=14))
fig.show()

print("""The number of participants of test group A is almost {:.0f} times higher than that of test group B.

The expected number of participants was 6,000. However, the number of users who actually participated was {}. 
Only {:.2f}% of the expected number of participants."""
      .format(rst_merge.query("group == 'A'")
              ['user_id'].nunique()/rst_merge.query("group == 'B'")
              ['user_id'].nunique(), rst_merge.user_id.nunique(), rst_merge.user_id.nunique()/6000*100))


#### Number of Participants and Participants Registered After December 21st, 2020

In [49]:
display(participants[participants.user_id.isin(past_date['user_id'])].query("ab_test == 'recommender_system_test'"))

print("""Even though the data initially consisted of users who signed up after December 21st, 2000, none were 
'recommender_system_test' members.""")



,user_id,group,ab_test


Even though the data initially consisted of users who signed up after December 21st, 2000, none were 
'recommender_system_test' members.


#### Participants' Region

In [ ]:
# checking what percetange of EU users were participants in the "recommender_system_test"

labels = ['EU Participants','EU Users'] # setting labels for slices 

# defining the data for the donut with the number of unique EU users and EU participants
values = [rst_merge.query("region == 'EU'").user_id.nunique(), filtered_users.
           query("region == 'EU'").user_id.nunique()-rst_merge.query
          ("region == 'EU'").user_id.nunique()]

# creating the donut, defining the hole, color and pulling the slice of EU participants
fig = go.Figure(data=[go.Pie(labels=labels, values=values, hole=.3, pull=[0, 0.2], 
                             marker=dict(colors=(px.colors.sequential.Emrld_r)))])

fig.update_layout(
    title_text="Percentage Of Participants from EU New Users") # setting the donut's title
fig.show()

print("""The intended test audience was 15% of the new users from the EU. However, in reality, the test audience 
was only {:.2f}%.""". format(rst_merge.query("region == 'EU'").user_id.nunique()/filtered_users.
                             query("region == 'EU'").user_id.nunique()*100))


In [50]:
rste_merge = rst_merge.query("region == 'EU'") # creating a data slice with only EU participants


In [ ]:
# checking how much of data was removed when filtering out none EU participants


labels = ["EU Participants", "Other Regions' Participants"] # setting labels for slices 

fig = make_subplots(rows=1, cols=2, specs=[[{'type':'domain'}, {'type':'domain'}]]) # creating subplots

# creating donut no.1 of number of unique users remaining and filtered out
fig.add_trace(go.Pie(labels=labels, values=[rste_merge['user_id'].nunique(), 
                        rst_merge['user_id'].nunique()-rste_merge['user_id'].nunique()], name="Users"),1, 1)


# creating donut no.2 of number of events remaining and filtered out
fig.add_trace(go.Pie(labels=labels, values=[rste_merge['event_name'].count(), 
                    rst_merge['event_name'].count()-rste_merge['event_name'].count()], name="Events"),1, 2)

# pulling out a slice, setting hole, color and hover info
fig.update_traces(hole=.3, hoverinfo="label+value+percent+name", 
                  marker=dict(colors=(px.colors.sequential.Teal)))

# setting the chart's title and the text in the donut holes
fig.update_layout(
    title_text="Percentage of Data Removed While Filtering",
    annotations=[dict(text='Users', x=0.19, y=0.5, font_size=15, showarrow=False),
                 dict(text='Events', x=0.815, y=0.5, font_size=15, showarrow=False)])

fig.show()

print("""As we saw in the previous section, a fraction of the "recommender_system_test" participants was not from 
the EU. At this stage they were filtered out and now {} participants remained.
""".format(rste_merge.user_id.nunique()))

#### Marketing Events

In [ ]:
# creating a gantt chart of the marketing events 

# creating the chart with the events, their starting and ending dates, setting the color and the hover data
fig = px.timeline(marketing_events.sort_values(by='start_dt'), x_start="start_dt", x_end="finish_dt", 
                  y="name", hover_data=['regions'], color_discrete_sequence=(px.colors.sequential.Teal_r))

fig.update_yaxes(autorange="reversed") # otherwise tasks are listed from the bottom up

fig.add_vline(x='2020-12-07', line_width=1, line_color="black", line_dash="dash") # setting the first vertical line

fig.add_vline(x='2020-12-21', line_width=1, line_color="red", line_dash="dash") # setting the second vertical line

fig.add_vline(x='2020-12-30', line_width=1, line_color="black", line_dash="dash") # setting the third vertical line

fig.add_vrect(x0='2020-12-07', x1='2020-12-21', line_width=0, fillcolor="green", opacity=0.1, 
              annotation_text="new users: December 7th-December 21st", annotation_position="top left", 
              annotation_textangle = 90) # setting the first colored area for the test sign up period

fig.add_vrect(x0='2020-12-21', x1='2020-12-30', line_width=0, fillcolor="red", opacity=0.1, 
              annotation_text="remaining test dates: January 1st, 2021", annotation_position="top left", 
              annotation_textangle = 90) # setting the second colored area for the remaining test period

fig.update_layout(hovermode='x unified', yaxis=dict(
        title='Marketing Event Name',
        titlefont_size=14,
        tickfont_size=12)) # creating a single hover label appear, describing one point per trace
fig.show()

print("""The 'Christmas&New Year Promo' for the EU and N.America regions coincides with the dates of the ab test 
and could have affected group members' actions - especially in the number of events leading up to promo and the 
conversion to the 'purchase' event funnel, since users probably received an incentive to buy a product.

The 'CIS New Year Gift Lottery' also coincides, but it was only for the CIS region (which was filtered out).""")


### Mid-Section Summary - A/B Test Results Analysis

In short, the A/B test results revealed that the test was not a success.

- The number of participants of test group A is almost 3 times higher than that of test group B;
- The expected number of participants was 6,000. However, the number of users who actually participated was 2788. Only 46.47% of the expected number of participants;
- Even though the data initially consisted of users who signed up after December 21st, 2000, none were 'recommender_system_test' members (i.e., they were either participants of the other test in the data or didn't participate in any test);
- The intended test audience was 15% of the new users from the EU. However, in reality, the test audience was only 6.72%;
- A fraction of the "recommender_system_test" participants was not from the EU - they were now filtered out, and 2594 participants remained;
- The 'Christmas&New Year Promo' for the EU and N.America regions coincides with the dates of the ab test and could have affected group members' actions - especially in the number of events leading up to promo and the conversion to the 'purchase' event funnel, since users probably received an incentive to buy a product;
- The 'CIS New Year Gift Lottery' also coincides, but it was only for the CIS region (which was filtered out).

### A/B Tests per Event - Significance Between the Proportions

Hereinafter, I will test the Null Hypothesis (H0) that group A's proportion is equal to that of group B.

If rejected, the Alternative Hypothesis (H1) is that group A's proportion is NOT equal to that of group B.

The A/B tests will be conducted per event, except for the "login" event since during this test better conversion was expected only for the following 3 events. Thus, we assume that 100% of members of both groups participated in this event. 

For this purpose, I will create a function applying the Z-test, in which we are checking two independent data groups and deciding whether a sample mean of two groups is equal or not. **Note, that the alpha will be set to 0.05 and then, using the The Bonferroni procedure (the Bonferroni correction), will be divided by 3 (m - the number of comparisons, i.e. events).**  

Next, I will apply the function to the number of users of the two groups in 3 of the events in the dataset. 

In [51]:
# creating a pivot table with the unique users of each event in each group, adding a row of all users and reindexing

exp_piv = rste_merge.pivot_table(columns="group", index="event_name", values='user_id', aggfunc='nunique').append(
    rste_merge.pivot_table(columns="group", values='user_id', aggfunc='nunique')).reset_index().rename(
    columns={'index': 'event'}).reindex([0, 2, 1, 3, 4]).reset_index(drop=True)

exp_piv # printing the pivot table 


group,event,A,B
0,login,1939,654
1,product_page,1265,367
2,product_cart,589,184
3,purchase,613,191
4,user_id,1939,655


Above, I created a table with the overall number of participants of each experiment group and the number of participants in each of the subsequent events. 

**It should be noted, that it is possible that due to a technical error a member of experiment group B has no "login" event. 
Our assumption should be that every member logs in. Thus, 100% of each experiment group should have that event.**

In [52]:
# creating a function for the z-test ab test for groups A and B of the "recommender_system_test"

alpha = 0.05 # significance level
bonferroni_alpha = alpha / 3 # using the bonferroni correction since three comparisons are being made

def test_ab(a,b,e):
    
    logs_a = exp_piv[exp_piv.event == "user_id"][a].iloc[0] # first group total visits
    logs_b = exp_piv[exp_piv.event == "user_id"][b].iloc[0] # second group total visits

    event_a = exp_piv[exp_piv.event == e][a].iloc[0] # first group specific events
    event_b = exp_piv[exp_piv.event == e][b].iloc[0] # second group specific events
    
    p1 = event_a/logs_a # events proportion in the first group

    p2 = event_b/logs_b # events proportion in the second group

    p_combined = (event_a + event_b) / (logs_a + logs_b) # success proportion in the combined dataset

    difference = p1 - p2 # the difference between the datasets' proportions

    # calculating the statistic in standard deviations of the standard normal distribution
    z_value = difference / mth.sqrt(p_combined * (1 - p_combined) * (1/logs_a + 1/logs_b))

    # setting up the standard normal distribution (mean 0, standard deviation 1)
    distr = st.norm(0, 1)  

    p_value = (1 - distr.cdf(abs(z_value))) * 2 # setting the p_value as the difference between 1 and z_value times 2

    print("""The share of users from group {} that had the {} event is {:.2%}.""".format(a, e, (event_a/logs_a)))
    print("""The share of users from group {} that had the {} event is {:.2%}.""".format(b, e, (event_b/logs_b)))
    print('p-value: {:.3f}'.format(p_value))

    if (p_value < bonferroni_alpha):
        print("""Conclusion: 
Reject the Null Hypothesis (H0) in regards to event {} using significance level (alpha) of {:.5f}.""".format(
            e, bonferroni_alpha))
    else:
        print("""Conclusion: 
We are unable reject the Null Hypothesis (H0) in regards to event {} using significance level 
(alpha) of {:.5f}.""".format(e, bonferroni_alpha))
        

In [53]:
# creating a loop implementing the z-test ab test function and printing the output 

events = ["product_page", "product_cart", "purchase"] # picking the events to be looped 

for e in events:
    test_ab("A", "B", e)
    print()
    print('-----------')
    print()

The share of users from group A that had the product_page event is 65.24%.
The share of users from group B that had the product_page event is 56.03%.
p-value: 0.000
Conclusion: 
Reject the Null Hypothesis (H0) in regards to event product_page using significance level (alpha) of 0.01667.

-----------

The share of users from group A that had the product_cart event is 30.38%.
The share of users from group B that had the product_cart event is 28.09%.
p-value: 0.269
Conclusion: 
We are unable reject the Null Hypothesis (H0) in regards to event product_cart using significance level 
(alpha) of 0.01667.

-----------

The share of users from group A that had the purchase event is 31.61%.
The share of users from group B that had the purchase event is 29.16%.
p-value: 0.240
Conclusion: 
We are unable reject the Null Hypothesis (H0) in regards to event purchase using significance level 
(alpha) of 0.01667.

-----------



#### Conclusions

For the first comparison, the product_page event, the p-value is lower than the alpha (0.01667) - this means that we **reject** the null hypothesis that the behavior the control group (A) and the test group (B) is equal in this event in the event funnel. Not only that, the share of users from group B is lower by more than 9% from that of group A. 

However, for two of the events (product_cart and and purchase) the p-value is higher than the alpha (0.01667) - this means that we **can't reject** the null hypothesis that the behavior the control group (A) and the test group (B) is equal in these two events in the event funnel.

## Overall Summary and Conclusions

### Summary

In this analysis, I performed the following steps before analyzing the A/B test results:


* Initial Exploration of the Datasets:
    - marketing_events
        * The dataset has 4 columns and 14 rows;
        * Most events are for specific regions, about 3 are for all regions. 
    - user_events
        * The dataset has 4 columns and 423,761 rows;
        * The "details" column has only 60,314 non null values - the nature of the null values will be examined 
           in the next section;
        * There are 58,703 users and 257,138 event dates and 4 types of events (the most common are "login" and
          "product_page";
        * There are only 4 prices: 4.99, 9.99, 99.99 and 499.99. 4.99 is by far the most frequent. 
    - users
        * The dataset has 4 columns and 58,703 rows (matches the number of users in the "user_events" dataset);
        * The most common region and sign up device are EU and Android (then PC), respectively;
        * Most users signed up on December 7th, the 14th and the 21st, 2020;
        * There are users who signed up after the exclusion date (December 21st).

    - participants
        * The dataset has 3 columns and 14,525 rows;
        * There are only 13,638 participants in the dataset (meaning there are a lot of users who aren't
          participants);
        * 3/4 of the participants (10,850 to be exact) participated in the "interface_eu_test" (not the test
          analyzed in this report);
        * This dataset is an early indication that the test was not performed as intended: 3,675 participants
          instead of 6,000 and it seems that some might be participants of the other test as well. 


* Data Preprocessing:
    * In the marketing_events dataset I converted the data types of two columns to datetime.
    * In the user_events dataset: 
        * Even though more than 85% of the values in the "details" column are null - there is a reasonable explanation for this and there is no need to remove them or fill them. The details is the priced paid for products users purchased and there are no null values for the "purchase" event;
        * The dates of the test are from December 7th, 2020, till January 1st, 2021 - so there is no need to remove         events from the dataset since they all fall within these dates.
    * In the users dataset there was an incorrect data type for the "first_date" column and there are users who signed up after December 21st, 2020.
    * In the participants dataset there were no duplicates or nulls and no data types to convert. 

    - Sign up date filtration 
        * Since there were users who signed up after December 21st, 2020, in the datasets - I saved their user ids in a table called "past_date" and filtered their ids from the users, user_events and participants datasets;
        * I checked that the filtration did not create anomalies in the distribution of values in the datasets - the distribution stayed the same only in fewer numbers;
        * Once these users were filtered out, the rows of each data set were as follows: 
            * users - 53,440;
            * user_events - 394,657 (56,170 non null values in the "details" column);
            * participants - 13,523.
            

* Exploratory Data Analysis:

    - Conversion
        * Overall, more than 60% of users who login to the store, visit a product's page and nearly 50% purchase
        the/a product. However, we can also see that it's possible to purchase a product directly from product
        page and skip the product cart since the purchase event has more users than the product cart event;
        * Most users logged in and view the product page and approximately the same amount of users visited the
        cart/made a purchase. It should also be noted that the groups are grossly uneven;
        * For the events performed by the participants of the "recommender_system_test", we can see that group A
        is significantly smaller than group B and that for both groups the trends of the overall users are the same. 
        
    - Events per User
        * Most users in the dataset had either 4, 6, 8 or 12 events between December 7th, 2020, and January 1st,
        2021 (actual last date is December 30th, 2020). Very few users had more than 12 events;
        * The participants of the "recommender_system_test" follow the same trends of all the users, with more
        than 800 members having 6 events. Overall, most users had no more than 12 events. 
        
    - Events per Day
        * The participants of the "recommender_system_test" follow the same trends of all the users, with more
        than 800 members having 6 events. Overall, most users had no more than 12 events; 
        * The distribution of events per day among the participants of the "recommender_system_test" is similar to
        the overall distribution, with one exception - no mini peak of December 24th. 
        
    - Overlapping test participants and filtering non-recommender_system_test participants
        * There were no participants who are members of both groups in either of the tests in the dataset;
        * There 887 participants who are members of both the "recommender_system_test" and the
        "interface_eu_test";
        * created new tables of each dataset with only the data pertaining to the participants of the
        "recommender_system_test" who participated in this test alone. The data filtered out is more than 95% of
        users and events. 
        
    - Additional Details
        * I merged all the data pertaining to the participants of the "recommender_system_test" into one dataset;
        * Most of the participants are from the EU but there are some participants from other regions among the
        participants;
        * Most participants signed up on December 7th, the 14th and the 21st. There were better sign up numbers
        leading to the 21st. Participants used mostly Android and PC to sign up;
        * Android is by far the device used the most to sign up - more than 1,200 participants (out of 2,788
        participants). Next is PC and iPhone while Mac is last. It is possible that most participants sign up "on
        the go" maybe they saw an advertisement, or that the interface is more user friendly.
        
        
* A/B Test Analysis:
    - The A/B Test Results
        - The number of participants of test group A is almost 3 times higher than that of test group B;
        - The expected number of participants was 6,000. However, the number of users who actually participated
        was 2788. Only 46.47% of the expected number of participants;
        - Even though the data initially consisted of users who signed up after December 21st, 2000, none were
        'recommender_system_test' members (i.e., they were either participants of the other test in the data or
        didn't participate in any test);
        - The intended test audience was 15% of the new users from the EU. However, in reality, the test audience
        was only 6.72%;
        - A fraction of the "recommender_system_test" participants was not from the EU - they were now filtered
        out, and 2594 participants remained;
        - The 'Christmas&New Year Promo' for the EU and N.America regions coincides with the dates of the ab test
        and could have affected group members' actions - especially in the number of events leading up to promo
        and the conversion to the 'purchase' event funnel, since users probably received an incentive to buy a
        product;
        - The 'CIS New Year Gift Lottery' also coincides, but it was only for the CIS region (which was filtered out).
    - Studying the Statistical Significance between the Proportions of the Experiment Groups
       * The Null Hypothesis (H0) is that group A's proportion is equal to that of group B;
       * The Alternative Hypothesis (H1) is that group A's proportion is NOT equal to that of group B;
       * The A/B tests were conducted per event, except for the "login" event;
       * A function was created to apply the Z-test (used to test hypotheses about the equality of proportions of
       populations);
       * The alpha was set to 0.05 and then, using the The Bonferroni procedure (the Bonferroni correction), was
       be divided by 3 (m - the number of comparisons, i.e. events).**          
        
**In short, the A/B test results revealed that the test was not a success and was not conducted as intended - contaminated by participants of another test, regions that are not EU, extremely unequal test groups and less than half of the expected number of test participants.**


### Conclusions

The behavior of the test group (B), the users who experienced the changes related to the introduction of an improved recommendation system, is similar to that of the members of the control group (A) in the product_cart and purchase events (the ones generating revenue).

The only event that where the behavior of the test group (B) was different to that of the control group (A) was the product_page event. However, their share was smaller than that of the control group.

**The changes related to the introduction of an improved recommendation system have no bearing on the users' experience and don't increase conversion into product page views (the product_page event), product card views (product_card) and purchases (purchase).**

**The conversion certainly did not increase by 10% (as was expected), and not only that, for the product_page event it DECREASED by more than 9%.** 